In [2]:
!git clone https://github.com/google/model_search.git

Cloning into 'model_search'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 203 (delta 6), reused 22 (delta 4), pack-reused 170
Receiving objects: 100% (203/203), 252.61 KiB | 14.86 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
%cd model_search
!pwd

/content/model_search
/content/model_search


In [4]:
!ls

CONTRIBUTING.md  LICENSE  model_search	README.md  requirements.txt  WORKSPACE


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 516.2MB 33kB/s 
     |████████████████████████████████| 133kB 50.0MB/s 
     |████████████████████████████████| 358kB 31.4MB/s 
     |████████████████████████████████| 2.8MB 47.3MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 460kB 55.4MB/s 
     |████████████████████████████████| 3.0MB 51.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=c3b309cf8d87fc6f1a015b6f72594ab50cff7dd58effdd55d2dfad098b8361a4
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=6f14d5c210ed76209ae01ea1ad22d1a7fdbcc3fd7f033ec0344e3a45fb9f4902
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
  Found existing insta

In [7]:
!protoc --python_out=./ model_search/proto/phoenix_spec.proto
!protoc --python_out=./ model_search/proto/hparam.proto
!protoc --python_out=./ model_search/proto/distillation_spec.proto
!protoc --python_out=./ model_search/proto/ensembling_spec.proto
!protoc --python_out=./ model_search/proto/transfer_learning_spec.proto


In [22]:
# change data path to conatin the processed data
data_path = "/content/drive/MyDrive/model_search/data/ProcessedData/"
!ls "/content/drive/MyDrive/model_search/data/ProcessedData/"

E003.expr.csv	E005.train.csv	E037.expr.csv	E116.train.csv
E003.test.csv	E005.valid.csv	E037.test.csv	E116.valid.csv
E003.train.csv	E006.expr.csv	E037.train.csv	E123.expr.csv
E003.valid.csv	E006.test.csv	E037.valid.csv	E123.test.csv
E004.expr.csv	E006.train.csv	E038.expr.csv	E123.train.csv
E004.test.csv	E006.valid.csv	E038.test.csv	E123.valid.csv
E004.train.csv	E007.expr.csv	E038.train.csv	GeneFile.windows.bed.zip
E004.valid.csv	E007.test.csv	E038.valid.csv
E005.expr.csv	E007.train.csv	E116.expr.csv
E005.test.csv	E007.valid.csv	E116.test.csv


In [49]:
# Concatenate and label data
import pandas as pd
REMC_ids = ["E003","E004","E005","E006","E007","E123","E116","E037","E038"]
df = pd.DataFrame()

# Not sure if label should go with id
label = 0;
for id in REMC_ids:
  file = data_path+id+".train.csv"
  temp = pd.read_csv(file,header=None)
  temp[0].values[:] = label
  df = df.append(temp)
  label = label + 1

# # Or if label should go with gene
# for id in REMC_ids:
#   file = data_path+id+".train.csv"
#   df = df.append(pd.read_csv(file,header=None))
# df[df.columns[0]] = df[df.columns[0]].str.extract(r'([_]{1}[^_]+[_]{1})')
# df[0] = pd.factorize(df[0])[0]

print(df)

         0  1  2  3  4  5
0        0  2  0  0  1  1
1        0  3  1  0  1  1
2        0  4  0  0  1  0
3        0  0  1  0  0  0
4        0  0  0  0  0  0
...     .. .. .. .. .. ..
1999995  8  0  0  0  1  0
1999996  8  1  0  1  1  0
1999997  8  1  0  3  1  1
1999998  8  1  0  1  0  0
1999999  8  2  0  0  0  1

[18000000 rows x 6 columns]


In [50]:
# Save dataframe as temporary csv
df.to_csv('tmp.csv',header=False,index=False)

In [51]:
# Use google model search to create trainer
import model_search
from model_search import constants
from model_search import single_trainer
from model_search.data import csv_data

trainer = single_trainer.SingleTrainer(
    data=csv_data.Provider(
        label_index=0,
        logits_dimension=2,
        record_defaults=[0,0,0,0,0,0],
        filename="tmp.csv"),
    spec='model_search/configs/dnn_config.pbtxt')

In [52]:
!mkdir /tmp/run_example

mkdir: cannot create directory ‘/tmp/run_example’: File exists


In [53]:
!rm -r /tmp/run_example/

In [54]:
import sys
from absl import app
sys.argv = sys.argv[:1]

try:
  app.run(lambda argv: None)
except:
  pass

In [55]:
# Use google model search with any number of trials
trainer.try_models(
    number_models=200,
    train_steps=5,
    eval_steps=1,
    root_dir="/tmp/run_example",
    batch_size=512,
    experiment_name="example",
    experiment_owner="model_search_user")

I0415 21:06:35.402992 139697096087424 metadata_store.py:93] MetadataStore with DB connection initialized
I0415 21:06:35.425943 139697096087424 oss_trainer_lib.py:290] creating directory: /tmp/run_example/tuner-1/1
I0415 21:06:35.427061 139697096087424 oss_trainer_lib.py:337] Tuner id: tuner-1
I0415 21:06:35.430899 139697096087424 oss_trainer_lib.py:338] Training with the following hyperparameters: 
I0415 21:06:35.434275 139697096087424 oss_trainer_lib.py:339] {'learning_rate': 8.7941194222574e-06, 'new_block_type': 'FULLY_CONNECTED_RESIDUAL_PROJECT_BATCHNORM', 'optimizer': 'rmsprop', 'initial_architecture_0': 'FULLY_CONNECTED_RESIDUAL_FORCE_MATCH_SHAPES_BATCHNORM', 'exponential_decay_rate': 0.7953411691736654, 'exponential_decay_steps': 2, 'gradient_max_norm': 4, 'dropout_rate': 0.40000001490116116, 'initial_architecture': ['FULLY_CONNECTED_RESIDUAL_FORCE_MATCH_SHAPES_BATCHNORM']}


INFO:tensorflow:TF_CONFIG environment variable: {'model_dir': '/tmp/run_example/tuner-1/1', 'session_master': ''}


I0415 21:06:35.435672 139697096087424 run_config.py:550] TF_CONFIG environment variable: {'model_dir': '/tmp/run_example/tuner-1/1', 'session_master': ''}


INFO:tensorflow:Using model_dir in TF_CONFIG: /tmp/run_example/tuner-1/1


I0415 21:06:35.440939 139697096087424 run_config.py:973] Using model_dir in TF_CONFIG: /tmp/run_example/tuner-1/1


...

I0415 21:12:33.577648 139697096087424 builder_impl.py:666] Assets added to graph.


INFO:tensorflow:No assets to write.


I0415 21:12:33.579406 139697096087424 builder_impl.py:461] No assets to write.


INFO:tensorflow:SavedModel written to: /tmp/run_example/tuner-1/5/saved_model/temp-1618521153/saved_model.pb


I0415 21:12:33.649114 139697096087424 builder_impl.py:426] SavedModel written to: /tmp/run_example/tuner-1/5/saved_model/temp-1618521153/saved_model.pb
I0415 21:12:33.658096 139697096087424 oss_trainer_lib.py:265] Evaluation results: {'accuracy': 0.46484375, 'auc_pr': 0.0, 'auc_roc': 1.0, 'loss': 3.2725117, 'num_parameters': 4192, 'global_step': 101}
I0415 21:12:33.669518 139697096087424 oss_trainer_lib.py:290] creating directory: /tmp/run_example/tuner-1/6


In [58]:
# Modify this cell to retrieve model data with the best accuracy
!ls /tmp/run_example/tuner-1/3

3.arch.txt			  model.ckpt-5.data-00000-of-00001
3.search_edge.txt		  model.ckpt-5.index
checkpoint			  model.ckpt-5.meta
graph.pbtxt			  replay_config.pbtxt
model.ckpt-0.data-00000-of-00001  saved_model
model.ckpt-0.index		  search_generator_0
model.ckpt-0.meta


In [60]:
# Show model summary
!cat /tmp/run_example/tuner-1/3/graph.pbtxt

Streaming output truncated to the last 5000 lines.
      type: DT_STRING
    }
  }
  attr {
    key: "Tshape"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "_output_shapes"
    value {
      list {
        shape {
          dim {
            size: 21
          }
        }
      }
    }
  }
}
node {
  name: "report_uninitialized_variables/boolean_mask/Reshape_1/shape"
  op: "Const"
  device: "/device:CPU:0"
  attr {
    key: "_output_shapes"
    value {
      list {
        shape {
          dim {
            size: 1
          }
        }
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 1
          }
        }
        int_val: -1
      }
    }
  }
}
node {
  name: "report_uninitialized_variables/boolean_mask/Reshape_1"
  op: "Reshape"
  input: "report_uninitialized_variables/LogicalNot"
  input: "report